In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
from google.colab import drive
drive.mount('/content/gdrive')
# RNN 은 입력데이터가 이전 데이터에 영향을 주는 데이터 처리에 적합한 구조이다.
# LSTM rnn에서 메모리를 잠시 기억하여 입력데이터를 다음노드에 추가하는 구조 

In [ ]:
def findFiles(path): return glob.glob(path)
print(findFiles('/content/gdrive/My Drive/Test/data/names/*.txt'))

In [ ]:
import unicodedata
import string
# RNN 은  hidden_layer에 이전결과를 받아서 사용할 수 있으므로 데이터가 
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD',s) 
      if unicodedata.category(c) != 'Mn'
      and c in all_letters
  )
print(unicodeToAscii('¿lusàrski'))

category_lines = {}
all_categories = []

def readLines(filename):
  lines = open(filename, encoding='utf-8').read().strip().split('\n')
  return [unicodeToAscii(line) for line in lines]

for filename in findFiles('/content/gdrive/My Drive/Test/data/names/*.txt'):
  category = os.path.splitext(os.path.basename(filename))[0]
  all_categories.append(category)
  lines = readLines(filename)
  category_lines[category] = lines

n_categories = len(all_categories)
print(category_lines['Korean'][:5])


In [ ]:
import torch

def letterToIndex(letter):
  return all_letters.find(letter)

def letterToTensor(letter):
  tensor = torch.zeros(1,n_letters)
  tensor[0][letterToIndex(letter)] = 1
  return tensor 

def lineToTensor(line):
  tensor=torch.zeros(len(line), 1, n_letters)
  for li, letter in enumerate(line):
    tensor[li][0][letterToIndex(letter)] = 1
  return tensor

print(letterToTensor('J'))
print(lineToTensor('Jones').size())


In [ ]:
import torch.nn as nn

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
    self.i2o = nn.Linear(input_size + hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim = 1)
  
  def forward(self, input, hidden):
    combined = torch.cat((input, hidden), 1)
    hidden = self.i2h(combined)
    output = self.i2o(combined)
    output = self.softmax(output)
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, self.hidden_size)  

In [ ]:
n_hidden = 128
rnn = RNN(n_letters ,n_hidden, n_categories)
input = lineToTensor('Albert')
hidden = torch.zeros(1,n_hidden)
output, next_hidden = rnn(input[0], hidden)
print(output)

def categoryFromOutput(output):
  top_n, top_i = output.topk(1)
  category_i = top_i[0].item()
  return all_categories[category_i], category_i

print(categoryFromOutput)

In [ ]:
import random

def randomChoice(l):
  return l[random.randint(0,len(l) - 1)]

def randomTrainingExample():
  category = randomChoice(all_categories)
  line = randomChoice(category_lines[category])
  category_tensor = torch.tensor([all_categories.index(category)], dtype = torch.long)
  line_tensor = lineToTensor(line)
  return category, line, category_tensor, line_tensor

for i in range(10):
  category, line, category_tensor, line_tensor = randomTrainingExample()
  print('category = ', category, '/ line = ', line)

criterion = nn.NLLLoss()
learning_rate = 0.005


In [ ]:
def train(category_tensor, line_tensor):
  hidden = rnn.initHidden()
  rnn.zero_grad()
  for i in range(line_tensor.size()[0]):
    output, hidden = rnn(line_tensor[i],hidden)
  
  loss = criterion(output, category_tensor)
  loss.backward()
  for p in rnn.parameters():
    p.data.add_(p.grad.data, alpha =-learning_rate)
  return output, loss.item()


In [ ]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000

current_loss = 0
all_losses = []

def timeSince(since):
  now = time.time()
  s = now - since
  m = math.floor(s/60)
  s -= m*60
  return '%dm %ds' % (m,s)

start = time.time()

for iter in range(1, n_iters + 1):
  category, line, category_tensor, line_tensor = randomTrainingExample()
  output, loss = train(category_tensor, line_tensor)
  current_loss += loss
  if iter % print_every == 0:
    guess, guess_i = categoryFromOutput(output)
    correct = '¿' if guess == category else '¿ (%s)' % category
    print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))
  if iter % plot_every == 0:
    all_losses.append(current_loss / plot_every)
    current_loss = 0

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

def evaluate(line_tensor):
  hidden = rnn.initHidden()
  for i in range(line_tensor.size()[0]):
    output, hidden = rnn(line_tensor[i], hidden)
  return output

for i in range(n_confusion):
  category, line, category_tensor, line_tensor = randomTrainingExample()
  output = evaluate(line_tensor)
  guess, guess_i = categoryFromOutput(output)
  category_i = all_categories.index(category)
  confusion[category_i][guess_i] +=1 

for i in range(n_categories):
  confusion[i] = confusion[i] / confusion[i].sum()

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

ax.set_xticklabels([' '] + all_categories, rotation=90)
ax.set_yticklabels([' '] + all_categories)

ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

plt.show()

def predict(input_line, n_prediction = 3):
  print('\n> %s' % input_line)
  with torch.no_grad():
    output = evaluate(lineToTensor(input_line))
    topv, topi = output.topk(n_prediction, 1, True)
    prediction = []
    for i in range(n_prediction):
      value = topv[0][i].item()
      category_index = topi[0][i].item()
      print('(%.2f) %s' % (value, all_categories[category_index]))
      prediction.append([value, all_categories[category_index]])

predict('Miles')
predict('Winston')
predict('Baker')
  # regression deep learinging example pytorch cinvilution